In [50]:
import requests
import json
import pandas as pd
from datetime import datetime as dt
from datetime import timezone
from dateutil import parser

# Show all columns when printing a pandas DataFrame
pd.options.display.max_columns = None

In [51]:
# Craetes a list of the fileds with a file as input
def read_fields(filename):
    fields = []
    fd = open(filename)
    lines = fd.readlines()
    for line in lines:
        fields.append("data."+line[:-1])
    fd.close()
    return fields

def remove_dot_data(fields):
    fields_no_data=[]
    for field in fields:
        field_no_data = field.replace("data.","")
        fields_no_data.append(field_no_data)
    return fields_no_data

def read_token(filename):
    fd = open(filename)
    line = fd.readline()
    # Remove the 'newline' character if found
    if line[len(line)-1] == '\n':
        line = line[:-1]
    fd.close()
    return line

In [53]:
# Where to get the authorization token
file_token="./token"

# Where to get the list of fields to retrieve
file_fields="./fields"

# Read the authorization token
token = read_token(file_token)
headers = {'Authorization': 'Bearer '+token,}

# Set the start(min) and end(max) dates to perform the query onto
min_date_human_readable = "Jun 6 10:00:00 UTC 2019"
max_date_human_readable = "Jun 6 10:59:00 UTC 2019"

# Get timestamps in miliseconds
min_date = str(int(parser.parse(min_date_human_readable).timestamp() * 1000))
max_date = str(int(parser.parse(max_date_human_readable).timestamp() * 1000))

# The query on lucene syntaxis
query =               "data.Type:analysis"
query = query + " AND  data.Status:Completed"
query = query + " AND  data.JobUniverse:5"

# Read the list of fields to retrieve from a file
fields = read_fields(file_fields)
fields_no_data = remove_dot_data(fields)

# Number of records to retreive
num_records = 100

In [54]:
print("Query:")
print(query)
print("Fields no data:")
print(fields_no_data)

data = {
        "size":num_records,
        "query":{
            "bool":{
                "filter":[
                    {"range":{"data.RecordTime":{"gte":min_date,"lte":max_date,"format":"epoch_millis"}}},
                    {"query_string":{
                        "analyze_wildcard":"true",
                        "query":query
                        }
                    }
                ]
            }
        },
        "_source":fields
}

data_string = json.dumps(data)

Query:
data.Type:analysis AND  data.Status:Completed AND  data.JobUniverse:5
Fields no data:
['CMSPrimaryPrimaryDataset', 'CMSPrimaryDataset', 'CMSPrimaryProcessedDataset', 'CMSPrimaryDataTier', 'CMSSite', 'CMS_TaskType', 'CRAB_UserHN', 'CoreHr', 'CpuTimeHr', 'ExitCode', 'RequestCpus', 'ScheddName']


In [55]:
# Send the query (It takes few seconds)
response = requests.get('https://monit-grafana.cern.ch/api/datasources/proxy/9014/_search', headers=headers, data=data_string)


In [56]:
# Get the data from the response
d = response.json()
# Create a pandas DataFrame with the data retreived
clean_records=[]
for record in d['hits']['hits']:
    clean_record= record['_source']['data']
    clean_records.append(clean_record)
df = pd.DataFrame(clean_records) 

In [57]:
# Print the DataFrame
df

,CMSPrimaryDataTier,CMSPrimaryPrimaryDataset,CMSPrimaryProcessedDataset,CMSSite,CMS_TaskType,CRAB_UserHN,CoreHr,CpuTimeHr,ExitCode,RequestCpus,ScheddName
0,MINIAOD,JetHT,Run2017C-31Mar2018-v1,T2_FR_CCIN2P3,cmsRun,paconnor,0.173611,0.137222,0,1,crab3@vocms0198.cern.ch
1,MINIAODSIM,QCD_Pt_470to600_TuneCP5_13TeV_pythia8,RunIIFall17MiniAODv2-PU2017_12Apr2018_94X_mc20...,T2_IT_Rome,cmsRun,paconnor,0.192500,0.049722,0,1,crab3@vocms0198.cern.ch
2,MINIAOD,JetHT,Run2017C-31Mar2018-v1,T2_US_Wisconsin,cmsRun,paconnor,0.133333,0.084167,0,1,crab3@vocms0198.cern.ch
3,MINIAOD,JetHT,Run2017C-31Mar2018-v1,T2_US_Wisconsin,cmsRun,paconnor,0.114722,0.074444,0,1,crab3@vocms0198.cern.ch
4,MINIAODSIM,QCD_Pt_470to600_TuneCP5_13TeV_pythia8,RunIIFall17MiniAODv2-PU2017_12Apr2018_94X_mc20...,T2_US_Caltech,cmsRun,paconnor,0.181667,0.056111,0,1,crab3@vocms0198.cern.ch
5,MINIAOD,JetHT,Run2017C-31Mar2018-v1,T2_US_UCSD,cmsRun,paconnor,0.118056,0.090833,0,1,crab3@vocms0198.cern.ch
6,MINIAOD,JetHT,Run2017C-31Mar2018-v1,T2_US_Wisconsin,cmsRun,paconnor,0.130556,0.076111,0,1,crab3@vocms0198.cern.ch
7,MINIAOD,SingleElectron,Run2016G-17Jul2018-v1,T2_US_Wisconsin,cmsRun,gmestdac,2.069167,1.804722,0,1,crab3@vocms0198.cern.ch
8,MINIAOD,SingleMuon,Run2018D-PromptReco-v2,T2_CH_CERN,cmsRun,youying,1.610833,1.583889,0,1,crab3@vocms0198.cern.ch
9,MINIAODSIM,QCD_Pt_170to300_TuneCUETP8M1_13TeV_pythia8,RunIISummer16MiniAODv3-PUMoriond17_94X_mcRun2_...,T2_ES_IFCA,cmsRun,paconnor,0.063889,0.009722,0,1,crab3@vocms0144.cern.ch
